In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report as clrp
kf = KFold(n_splits=5)
df = pd.read_csv('fake_job_postings.csv')
df.head(2)

In [ ]:
lemmatizer = WordNetLemmatizer() 
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
stop = stopwords.words('english')

def lemmatize_text1(text):
    return [lemmatizer.lemmatize(w,pos="v") for w in w_tokenizer.tokenize(text)]
def lemmatize_text2(text):
    return [lemmatizer.lemmatize(w,pos="n") for w in text]
df_desc = df.description.fillna(' ').combine(df.requirements.fillna(' '), lambda x,y: x+y)
df_desc= df_desc.str.replace('[^\w\s]','') #remove punctuation
df_desc= df_desc.str.replace('\d+', '') #remove numbers 
df_desc= df_desc.apply(lambda x: " ".join(x.lower() for x in str(x).split())) #lowercase
df_desc= df_desc.apply(lemmatize_text1)
df_desc= df_desc.apply(lemmatize_text2)
df_desc= df_desc.apply(lambda x: " ".join(x for x in x if x not in stop)) #remove stopwords

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(2,3),max_features=200)
sparse_matrix = vectorizer.fit_transform(df_desc)

In [ ]:
sparse_matrix = sparse_matrix.todense()
sparse_matrix.shape

In [ ]:
df=df.drop(columns=['job_id','title','location','department','company_profile','description','requirements','industry','function'])

In [ ]:
df = df.fillna('empty')
df['salary_range'] = df.salary_range.apply(lambda x : 0 if x== 'empty' else 1)
df['benefits'] = df.benefits.apply(lambda x : 0 if x == 'empty' else 1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
onehotencoder = OneHotEncoder(sparse=False,drop = ['empty'])
df1=onehotencoder.fit_transform(np.array(df.employment_type).reshape(-1,1))
df1 = pd.DataFrame(df1)
#df1 = df1.drop(columns = 0)
for i in range(df1.shape[1]):
    df['emp_type'+ str(i)] = df1.iloc[:,i]
df = df.drop(columns = 'employment_type')

In [ ]:
df1=onehotencoder.fit_transform(np.array(df.required_experience).reshape(-1,1))
df1 = pd.DataFrame(df1)
#df1 = df1.drop(columns = 0)
for i in range(df1.shape[1]):
    df['req_exp'+ str(i)] = df1.iloc[:,i]
df = df.drop(columns = 'required_experience')

In [ ]:
df1=onehotencoder.fit_transform(np.array(df.required_education).reshape(-1,1))
df1 = pd.DataFrame(df1)
#df1 = df1.drop(columns = 0)
for i in range(df1.shape[1]):
    df['req_edu'+ str(i)] = df1.iloc[:,i]
df = df.drop(columns = 'required_education')

In [ ]:
df.head(5)

In [ ]:
ytrain_matrix = df.fraudulent
xtrain_matrix = df.drop(columns = 'fraudulent')

In [ ]:
xtrain_matrix = np.array(xtrain_matrix)
xtrain_matrix = np.concatenate((xtrain_matrix,sparse_matrix), axis=1)
xtrain_matrix.shape

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report as clrp
kf = KFold(n_splits=5)

In [ ]:
from sklearn.svm import LinearSVC
classifier= LinearSVC(random_state=0, tol=1e-5) 
for train_index, test_index in kf.split(xtrain_matrix,ytrain_matrix):
    classifier.fit(xtrain_matrix[train_index],ytrain_matrix[train_index])
    ypred=classifier.predict(xtrain_matrix[test_index])
    ytestt=ytrain_matrix[test_index]
    print(clrp(ytestt,ypred))

In [ ]:
from xgboost import XGBClassifier
classifier2 = XGBClassifier()
for train_index, test_index in kf.split(xtrain_matrix,ytrain_matrix):
    classifier2.fit(xtrain_matrix[train_index],ytrain_matrix[train_index])
    ypred=classifier2.predict(xtrain_matrix[test_index])
    ytestt=ytrain_matrix[test_index]
    print(clrp(ytestt,ypred))